In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import gurobipy as gp
from gurobipy import GRB

In [4]:

data_dir = Path.cwd() / "Data"

df_genTech = pd.read_csv(data_dir / "TechnicalDataofGeneratingUnits-Table1.csv")
df_gen     = pd.read_csv(data_dir / "CostsandInitialStateofGeneratingUnits-Table2.csv")
df_load    = pd.read_csv(data_dir / "LoadProfile-Table3.csv")

df_demand  = pd.read_csv(data_dir / "NodeLocationandDistributionoftheTotalSystemDemand-Table4.csv")
df_demand["Percent"] = df_demand["Percent"]/100

# ---- renewables (your file is comma-delimited) ----
df_renew = pd.read_csv(data_dir / "scen_zone1.out")
df_renew = df_renew.apply(pd.to_numeric, errors="coerce")
df_renew = df_renew.iloc[:, 1:]


U_d = pd.read_csv(data_dir / "demand_utility_prices_24h.csv", index_col="Hour")

#U_d = [30.5, 31.2, 32.0, 32.8, 33.4,
# 33.9, 34.3, 34.8, 35.2, 35.9,
# 36.5, 37.1, 37.8, 38.6, 39.5,
# 40.3, 41.2]

In [ ]:
# If it's one big comma-separated string per row, split it
if df_renew.shape[1] == 1:
    df_renew = df_renew[0].astype(str).str.split(",", expand=True)

# Convert everything to numeric (so mean() works)
df_renew = df_renew.apply(pd.to_numeric, errors="coerce")

# Auto column names: C0, C1, C2, ...
df_renew.columns = [f"C{i}" for i in range(df_renew.shape[1])]

# Treat first column as time/index, rest as renewables
renew_cols = df_renew.columns[1:]          # all renewable columns
# If you ONLY want 6 series like before, use:
# renew_cols = df_renew.columns[1:7]

# Mean across renewables
df_renew["mean"] = df_renew[renew_cols].mean(axis=1)

# Plot all renewable series
plt.plot(df_renew[renew_cols])
plt.show()

# Plot mean
plt.plot(df_renew["mean"])
plt.show()

# Your “bus plots” style (first 6 renewable columns)
bus_cols = list(df_renew.columns[1:7])  # C1..C6
bus_labels = ["Bus 3 Wind","Bus 5 Wind","Bus 7 Wind","Bus 16 Wind","Bus 21 Wind","Bus 23 Wind"]

for col, label in zip(bus_cols, bus_labels):
    plt.plot(df_renew[col])
    plt.legend([label])
    plt.show()

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Initialize Gurobi model
model = gp.Model('Copperplate')

# Time of day:
time = 12

#Constants
cap_re = 200 #Capacity on the renewable generators
N_d = len(df_demand)
N_gen = len(df_genTech) # Total number of generators
N_ren = 6 # Total number of renewable generators

# Variables 
Pgen = model.addVars(N_gen, vtype=GRB.CONTINUOUS, name="P_gen")
Pd   = model.addVars(N_d,   vtype=GRB.CONTINUOUS, name="P_demand")
Pw   = model.addVars(N_ren, vtype=GRB.CONTINUOUS, name="P_wind")

# Power balance constraint
balance = model.addConstr(
    gp.quicksum(Pgen[gen] for gen in range(N_gen))
    + gp.quicksum(Pw[ren] for ren in range(N_ren))
    - gp.quicksum(Pd[d] for d in range(N_d)) == 0,
    name="Power Balance"
)

# Demand Capacity Constraints
# (use iloc for time row; pick the load column that exists)
load_col = next(col for col in ["Demand","demand","Load","load","SystemLoad","Pload","P_Load"] if col in df_load.columns)
total_demand = float(df_load.iloc[time][load_col])

Dmax = {d: float(df_demand.loc[d, "Percent"]) * total_demand for d in range(N_d)}  # for printing later

model.addConstrs((Pd[d] <= Dmax[d] for d in range(N_d)), name="Demand Max Capacity")
model.addConstrs((Pd[d] >= 0 for d in range(N_d)),      name="Demand Min Capacity")

# Generator Capacity Constraints
Pmax = {g: float(df_genTech.loc[g, "Pmax"]) for g in range(N_gen)}  # for printing later
model.addConstrs((Pgen[g] <= Pmax[g] for g in range(N_gen)), name="Generator Max Capacity")
model.addConstrs((Pgen[g] >= 0 for g in range(N_gen)), name="Generator Min Capacity")

# Renewable Capacity Constraints
wind_cols = ["V1","V2","V3","V4","V5","V6"]
Wmax = {w: float(df_renew.iloc[time, w]) * cap_re for w in range(N_ren)}      # for printing later
model.addConstrs((Pw[w] <= Wmax[w] for w in range(N_ren)), name="Renewable Max Capacity")
model.addConstrs((Pw[w] >= 0 for w in range(N_ren)),       name="Renewable Min Capacity")


# Objective function
model.setObjective(sum(Pd[d] * U_d.iloc[time,d] for d in range(N_d))
                 - sum(Pgen[gen] * df_gen.loc[gen,"Ci"] for gen in range(N_gen))
                 - sum(Pw[ren] * 0 for ren in range(N_ren)), sense=GRB.MAXIMIZE)

model.optimize()

# -----------------------------
# RESULTS (simple)
# -----------------------------
# Market-clearing price is the dual of the power balance constraint (up to sign convention).

price = abs(balance.getAttr("Pi"))
test = balance.getAttr("Pi")
Pgen_sol = {g: Pgen[g].X for g in range(N_gen)}
Pw_sol   = {w: Pw[w].X for w in range(N_ren)}
Pd_sol   = {d: Pd[d].X for d in range(N_d)}

cost = sum(float(df_gen.loc[g, "Ci"]) * Pgen_sol[g] for g in range(N_gen))  # €/h
welfare = model.ObjVal                                                     # €/h

print(f"Hour {time} | Price={price:.2f} €/MWh | Cost={cost:.2f} €/h | Welfare={welfare:.2f} €/h")

# Optional tables for reporting
display(pd.DataFrame({"Pgen": Pgen_sol, "Pmax": Pmax, "Ci": df_gen["Ci"].astype(float).to_dict()}))
display(pd.DataFrame({"Pw": Pw_sol, "Wmax": Wmax}))
display(pd.DataFrame({"Pd": Pd_sol, "Dmax": Dmax, "Bid": {d: float(U_d.iloc[time,d]) for d in range(N_d)}}))

# 1) Print df_demand (full table)
print(total_demand)

# 2) Sum of Pmax (from df_genTech)
print("Sum of Pmax =", df_genTech["Pmax"].sum())



Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 71 rows, 35 columns and 105 nonzeros
Model fingerprint: 0x9bd1d192
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 70 rows and 9 columns
Presolve time: 0.01s
Presolved: 1 rows, 26 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.7679965e+04   1.827806e+02   0.000000e+00      0s
       1    8.6101520e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  8.610151996e+04
Hour 12 | Price=10.89 €/MWh | Cost=11578.45 €/h | Welfare=86101.52 €/h


,Pgen,Pmax,Ci
0,0.000000,152.0,13.32
1,0.000000,152.0,13.32
2,0.000000,350.0,20.70
3,0.000000,591.0,20.93
4,0.000000,60.0,26.11
5,155.000000,155.0,10.52
6,155.000000,155.0,10.52
7,400.000000,400.0,6.02
8,400.000000,400.0,5.47
9,300.000000,300.0,0.00


,Pw,Wmax
0,155.940883,155.940883
1,29.290732,29.290732
2,125.294395,125.294395
3,153.164290,153.164290
4,143.348758,143.348758
5,148.691188,148.691188


,Pd,Dmax,Bid
0,95.683050,95.683050,32.635
1,85.611150,85.611150,33.384
2,158.632425,158.632425,34.240
3,65.467350,65.467350,35.096
4,62.949375,62.949375,35.738
5,120.862800,120.862800,36.273
6,110.790900,110.790900,36.701
7,151.078500,151.078500,37.236
8,153.596475,153.596475,37.664
9,171.222300,171.222300,38.413


2517.975
Sum of Pmax = 3375


In [ ]:
# -----------------------------
# SIMPLE KKT CHECK (text labels)
# -----------------------------
tol = 1e-6
bid = {d: float(U_d.iloc[time,d]) for d in range(N_d)}  # <-- define bids from your data

print("\nKKT check (generators):")
for g in range(N_gen):
    Pg = Pgen_sol[g]
    mc = float(df_gen.loc[g, "Ci"])
    if Pg < tol:
        print(f"G{g}: rejected -> expect price <= Ci ({price:.2f} <= {mc:.2f})")
    elif Pg > Pmax[g] - tol:
        print(f"G{g}: at cap -> expect price >= Ci ({price:.2f} >= {mc:.2f})")
    else:
        print(f"G{g}: marginal -> expect price ≈ Ci ({price:.2f} ≈ {mc:.2f})")

print("\nKKT check (demand blocks):")
for d in range(N_d):
    x = Pd_sol[d]
    b = bid[d]
    if x < tol:
        print(f"D{d}: rejected -> expect price >= Bid ({price:.2f} >= {b:.2f})")
    elif x > Dmax[d] - tol:
        print(f"D{d}: at cap -> expect price <= Bid ({price:.2f} <= {b:.2f})")
    else:
        print(f"D{d}: marginal -> expect price ≈ Bid ({price:.2f} ≈ {b:.2f})")

In [ ]:
# show generators with smallest Pmax
pmax_series = df_genTech["Pmax"].astype(float)
print("Min Pmax:", pmax_series.min(), "Max Pmax:", pmax_series.max())
print("Count Pmax <= 1e-6:", (pmax_series <= 1e-6).sum())

# find the first few positive-cap generators and their costs
tmp = pd.DataFrame({
    "Pmax": df_genTech["Pmax"].astype(float),
    "Ci": df_gen["Ci"].astype(float)
})
tmp = tmp.sort_values("Ci")
display(tmp.head(15))


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

# visual settings
strip_h = 0.6          # height used to visualize any MC=0 block
tol_mc  = 1e-9

# build blocks: (mc, cap, kind, label)
wind_blocks = [(0.0, float(Wmax[w]), "wind", f"W{w+1}") for w in range(N_ren) if float(Wmax[w]) > 0]
conv_blocks = [(float(df_gen.loc[g, "Ci"]), float(Pmax[g]), "conv", f"G{g}") for g in range(N_gen) if float(Pmax[g]) > 0]
conv_blocks.sort(key=lambda x: x[0])

Q = float(sum(Pd_sol.values()))
p = float(price)

# unique colors for nonzero-cost conventional
nonzero_conv = [b for b in conv_blocks if b[0] > tol_mc]
cmap = cm.get_cmap("tab20", max(len(nonzero_conv), 1))
color_map = {lab: cmap(i) for i, (_, _, _, lab) in enumerate(nonzero_conv)}

fig, ax = plt.subplots(figsize=(12, 6), facecolor="white")
ax.set_facecolor("white")

cum = 0.0

# wind (MC=0) -> visible strip
for mc, cap, kind, lab in wind_blocks:
    ax.bar(cum, strip_h, width=cap, align="edge",
           color="white", edgecolor="tab:green", linewidth=2,
           hatch="///", alpha=1.0)
    cum += cap

# conventional blocks
for mc, cap, kind, lab in conv_blocks:
    if mc <= tol_mc:
        # zero-cost conventional -> visible strip (different hatch/color)
        ax.bar(cum, strip_h, width=cap, align="edge",
               color="white", edgecolor="dimgray", linewidth=2,
               hatch="\\\\\\", alpha=1.0)
    else:
        # normal conventional -> colored block at height = mc
        ax.bar(cum, mc, width=cap, align="edge",
               color=color_map[lab], edgecolor="white", linewidth=1.2, alpha=0.95)
    cum += cap

# clearing lines
ax.axvline(Q, color="black", linestyle="--", linewidth=2)
ax.axhline(p, color="tab:red", linestyle="--", linewidth=2.5)


# cosmetics
ax.set_title(f"Merit order curve (hour {time})", fontsize=12, pad=12)
ax.set_xlabel("Cumulative capacity (MW)", fontsize=12)
ax.set_ylabel("Marginal cost (€/MWh)", fontsize=12)
ax.grid(True, axis="y", alpha=0.25)
ax.set_xlim(0, max(cum, Q) * 1.02)
ax.set_ylim(0, max(p * 1.35, max([mc for mc,_,_,_ in conv_blocks] + [5]) * 1.1, 5))

# legend (with numbers included)
legend_items = [
    Patch(facecolor="white", edgecolor="tab:green", hatch="///",
          label="Wind (MC=0 shown as strip)"),
    Patch(facecolor="white", edgecolor="dimgray", hatch="\\\\\\",
          label="Zero-cost conventional (MC=0 shown as strip)"),
    Line2D([0],[0], color="tab:red", linestyle="--", linewidth=2.5,
           label=f"Clearing price = {p:.2f} €/MWh"),
    Line2D([0],[0], color="black", linestyle="--", linewidth=2.0,
           label=f"Clearing quantity = {Q:.1f} MW"),
]

ax.legend(handles=legend_items, loc="upper left", frameon=True, framealpha=0.95)


plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# -----------------------------
# Nord Pool–style aggregated bid curves (with "final step" padding for supply)
# Assumes you already have:
#   price, Pd_sol
#   Wmax (dict), Pmax (dict), Dmax (dict)
#   df_gen["Ci"], U_d, N_gen, N_ren, N_d, time
# -----------------------------

# --- 1) Build supply blocks (offer price, volume) ---
supply = [(0.0, float(Wmax[w])) for w in range(N_ren)] + \
         [(float(df_gen.loc[g, "Ci"]), float(Pmax[g])) for g in range(N_gen)]
supply = [(p, q) for (p, q) in supply if q > 1e-9]
supply.sort(key=lambda x: x[0])  # increasing offer price

# --- 2) Build demand blocks (bid price, volume) ---
demand = [(float(U_d[d]), float(Dmax[d])) for d in range(N_d)]
demand = [(b, q) for (b, q) in demand if q > 1e-9]
demand.sort(key=lambda x: x[0], reverse=True)  # decreasing bid

# --- 3) Step arrays: SUPPLY ---
xs, ys = [0.0], []
cum = 0.0
for ps, qs in supply:
    ys.append(ps)
    cum += qs
    xs.append(cum)
ys_step = [ys[0]] + ys  # match length of xs

# Pad right side so last supply step is visibly horizontal (visual only)
x_pad = xs[-1] * 0.03              # 3% extra width
xs_plot = xs + [xs[-1] + x_pad]
ys_plot = ys_step + [ys_step[-1]]

# --- 4) Step arrays: DEMAND (no extension; ends at total demand cap) ---
xd, yd = [0.0], []
cum = 0.0
for bd, qd in demand:
    yd.append(bd)
    cum += qd
    xd.append(cum)
yd_step = [yd[0]] + yd  # match length of xd

# --- 5) Clearing point from your optimization ---
Q_star = float(sum(Pd_sol.values()))
p_star = float(price)

# --- 6) Plot ---
fig, ax = plt.subplots(figsize=(11, 6), facecolor="white")
ax.set_facecolor("white")

# Supply (sell)
ax.step(xs_plot, ys_plot, where="post", linewidth=2.5, color="DeepSkyBlue",
        label="Aggregated supply (sell)")
ax.fill_between(xs_plot, 0, ys_plot, step="post", alpha=0.12, color="DeepSkyBlue")

# Demand (buy)
ax.step(xd, yd_step, where="post", linewidth=2.5, color="tab:red",
        label="Aggregated demand (buy)")
ax.fill_between(xd, 0, yd_step, step="post", alpha=0.10, color="tab:red")

# Clearing lines + point
ax.axvline(Q_star, linestyle="--", linewidth=1, color="black")
ax.axhline(p_star, linestyle="--", linewidth=1, color="Plum")
ax.plot([Q_star], [p_star], marker="o", markersize=6)

# Titles/labels
ax.set_title(f"Aggregated bid curves — 1 hour {time}", fontsize=14, pad=10)
ax.set_xlabel("Cumulative volume (MW)")
ax.set_ylabel("Price (€/MWh)")
ax.grid(True, alpha=0.25)

# Limits (show demand bids + padded supply end)
max_bid = max(b for b, q in demand) if len(demand) else 0
ax.set_ylim(0, max(max_bid * 1.05, p_star * 1.6, 10))
ax.set_xlim(0, xs_plot[-1])

# Legend with clearing values included
handles, labels = ax.get_legend_handles_labels()
handles += [
    Line2D([0],[0], color="Plum", linestyle="--", linewidth=2,
           label=f"Clearing price = {p_star:.2f} €/MWh"),
    Line2D([0],[0], color="black", linestyle="--", linewidth=2,
           label=f"Clearing quantity = {Q_star:.1f} MW"),
]
ax.legend(handles=handles, loc="upper right", frameon=True, framealpha=0.95,
          handlelength=3.0, handletextpad=0.8, labelspacing=0.6)

plt.tight_layout()
plt.show()
